In [1]:
# Description: Calculates correlations between daily vorticity terms.
#
# Author:      André Palóczy
# E-mail:      paloczy@gmail.com
# Date:        April/2020

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from xarray import open_dataset
from pandas import Series
from scipy.special import erfinv

In [3]:
def crosscorr(x, y, nblks, maxlags=0, overlap=0, onesided=False, verbose=True):
    """
    Lag-N cross correlation averaged with Welch's Method.
    Parameters
    ----------
    x, y     : Arrays of equal length.
    nblks    : Number of blocks to average cross-correlation.
    maxlags  : int, default (0) calculates te largest possible number of lags,
               i.e., the number of points in each chunk.
    overlap  : float, fraction of overlap between consecutive chunks. Default 0.
    onesided : Whether to calculate the cross-correlation only at
               positive lags (default False). Has no effect if
               x and y are the same array, in which case the
               one-sided autocorrelation function is calculated.

    Returns
    ----------
    crosscorr : float array.
    """
    if x is y:
        auto = True
    else:
        auto = False
    x, y = np.array(x), np.array(y)
    nx, ny = x.size, y.size
    assert x.size==y.size, "The series must have the same length"

    nblks, maxlags = int(nblks), int(maxlags)
    ni = int(nx/nblks)               # Number of data points in each chunk.
    dn = int(round(ni - overlap*ni)) # How many indices to move forward with
                                     # each chunk (depends on the % overlap).

    if maxlags==0:
        if verbose:
            print("Maximum lag was not specified. Accomodating it to block size (%d)."%ni)
        maxlags = ni
    elif maxlags>ni:
        if verbose:
            print("Maximum lag is too large. Accomodating it to block size (%d)."%ni)
        maxlags = ni

    if onesided:
        lags = range(maxlags+1)
    else:
        lags = range(-maxlags, maxlags+1)

    # Array that will receive cross-correlation of each block.
    xycorr = np.zeros(len(lags))

    n=0
    il, ir = 0, ni
    while ir<=nx:
        xn = x[il:ir]
        yn = y[il:ir]

        # Calculate cross-correlation for current block up to desired maximum lag - 1.
        xn, yn = map(Series, (xn, yn))
        xycorr += np.array([xn.corr(yn.shift(periods=lagn)) for lagn in lags])

        il+=dn; ir+=dn
        n+=1

    # pandas.Series.corr(method='pearson') -> pandas.nanops.nancorr() ...
    # -> pandas.nanops.get_corr_function() -> np.corrcoef -> numpy.cov(bias=False as default).
    # So np.corrcoef() returns the UNbiased correlation coefficient by default
    # (i.e., normalized by N-k instead of N).

    xycorr /= n    # Divide by number of blocks actually used.
    ncap = nx - il # Number of points left out at the end of array.

    if verbose:
        print("")
        if ncap==0:
            print("No data points were left out.")
        else:
            print("Left last %d data points out (%.1f %% of all points)."%(ncap,100*ncap/nx))
        print("Averaged %d blocks, each with %d lags."%(n,maxlags))
        if overlap>0:
            print("Intended %d blocks, but could fit %d blocks, with"%(nblks,n))
            print('overlap of %.1f %%, %d points per block.'%(100*overlap,dn))
        print("")

    lags = np.array(lags)
    if auto and not onesided:
        fo = np.where(lags==0)[0][0]
        xycorr[fo+1:] = xycorr[fo+1:] + xycorr[:fo]
        lags = lags[fo:]
        xycorr = xycorr[fo:]

    fgud=~np.isnan(xycorr)

    return lags[fgud], xycorr[fgud]


def Tdecorr(Rxx, M=None, dtau=1., verbose=False):
    """
    USAGE
    -----
    Td = Tdecorr(Rxx)

    Computes the integral scale Td (AKA decorrelation scale, independence scale)
    for a data sequence with autocorrelation function Rxx. 'M' is the number of
    lags to incorporate in the summation (defaults to all lags) and 'dtau' is the
    lag time step (defaults to 1).

    The formal definition of the integral scale is the total area under the
    autocorrelation curve Rxx(tau):

    /+inf
    Td = 2 * |     Rxx(tau) dtau
    /0

    In practice, however, Td may become unrealistic if all of Rxx is summed
    (e.g., often goes to zero for data dominated by periodic signals); a
    different approach is to instead change M in the summation and use the
    maximum value of the integral Td(t):

    /t
    Td(t) = 2 * |     Rxx(tau) dtau
    /0

    References
    ----------
    e.g., Thomson and Emery (2014),
    Data analysis methods in physical oceanography,
    p. 274, equation 3.137a.

    Gille lecture notes on data analysis, available
    at http://www-pord.ucsd.edu/~sgille/mae127/lecture10.pdf
    """
    Rxx = np.asanyarray(Rxx)
    C0 = Rxx[0]
    N = Rxx.size # Sequence size.

    # Number of lags 'M' to incorporate in the summation.
    # Sum over all of the sequence if M is not chosen.
    if not M:
        M = N

    # Integrate the autocorrelation function.
    Td = np.zeros(M)
    for m in range(M):
        Tdaux = 0.
        for k in range(m-1):
            Rm = (Rxx[k] + Rxx[k+1])/2. # Midpoint value of the autocorrelation function.
            Tdaux = Tdaux + Rm*dtau # Riemann-summing Rxx.

        Td[m] = Tdaux

    # Normalize the integral function by the autocorrelation at zero lag
    # and double it to include the contribution of the side with
    # negative lags (C is symmetric about zero).
    Td = (2./C0)*Td

    if verbose:
        print("")
        print("Theoretical integral scale --> 2 * int 0...+inf [Rxx(tau)] dtau: %.2f."%Td[-1])
        print("")
        print("Maximum value of the cumulative sum: %.2f."%Td.max())

    return Td


def Tdecorrw(x, nblks=30, ret_median=True, verbose=True):
    """
    USAGE
    -----
    Ti = Tdecorrw(x, nblks=30, ret_median=True, verbose=True)

    'Ti' is the integral timescale calculated from the
    autocorrelation function calculated for variable 'x'
    block-averaged in 'nblks' chunks.
    """
    x = np.array(x)
    dnblkslr = round(nblks/2)

    tis = [Tdecorr(crosscorr(x, x, nblks=n, verbose=verbose)[1]).max() for n in range(nblks-dnblkslr, nblks+dnblkslr+1)]
    tis = np.ma.masked_invalid(tis)

    if verbose:
        print("========================")
        print(tis)
        print("========================")
        p1, p2, p3, p4, p5 = map(np.percentile, [tis]*5, (10, 25, 50, 75, 90))
        print("--> 10 %%, 25 %%, 50 %%, 75 %%, 90 %% percentiles for Ti:  %.2f,  %.2f,  %.2f,  %.2f,  %.2f."%(p1, p2, p3, p4, p5))
        print("------------------------")

    if ret_median:
        return np.median(tis)
    else:
        return tis


def rsig(ndof_eff, alpha=0.95):
	"""
	USAGE
	-----
	Rsig = rsig(ndof_eff, alpha=0.95)

	Computes the minimum (absolute) threshold value 'rsig' that
	the Pearson correlation coefficient r between two normally-distributed
	data sequences with 'ndof_eff' effective degrees of freedom has to have
	to be statistically significant at the 'alpha' (defaults to 0.95)
	confidence level.

	For example, if rsig(ndof_eff, alpha=0.95) = 0.2 for a given pair of
	NORMALLY-DISTRIBUTED samples with a correlation coefficient r>0.7, there
	is a 95 % chance that the r estimated from the samples is significantly
	different from zero. In other words, there is a 5 % chance that two random
	sequences would have a correlation coefficient higher than 0.7.

	OBS: This assumes that the two data series have a normal distribution.

	Translated to Python from the original matlab code by Prof. Sarah Gille
	(significance.m), available at http://www-pord.ucsd.edu/~sgille/sio221c/

	References
	----------
	Gille lecture notes on data analysis, available
	at http://www-pord.ucsd.edu/~sgille/mae127/lecture10.pdf

	Example
	-------
	TODO
	"""
	rcrit_z = erfinv(alpha)*np.sqrt(2./ndof_eff)

	return rcrit_z

In [4]:
plt.close('all')

head_data = "../../data_reproduce_figs/"
terms = ['Ibetav', 'Icurlvdiff', 'Icurlhdiff', 'Istretchp', 'Ires', 'Icurlnonl']
segments = ['Amundsen-Bellingshausen', 'WAP', 'Weddell', 'W-EA', 'E-EA', 'Ross']

term_label = dict(Ibetav=r"$-\beta V$", Icurlvdiff=r"VVIS$_\xi$", Icurlhdiff=r"HVIS$_\xi$", Istretchp=r"$-fw_I$", Ires=r"-$\zeta_t$", Icurlnonl=r"-NONL$_\xi$", Ierrcor=r"-ERRCOR")

# Circumpolar circulation terms.
fname = head_data+'circulation_terms_circumpolar.nc'
ds = open_dataset(fname)

# Line plot with 2005-2009 time series of all circumpolar terms.
t = ds['t']

if False:
	Ti = dict()
	for term in terms:
		plt.plot(t, ds[term].values, label=term_label[term])
		Ti.update({term:Tdecorrw(ds[term].values)})
	plt.legend()

	EDoF = 365
	rmin = rsig(EDoF, alpha=0.99)
	print("Minimum statistically significant correlation coeff at 0.95 CL: %.2f"%rmin)

if False:
	F = ds['Icurlvdiff']
	R_TSB = -ds['Ibetav'] - ds['Istretchp']
	R_tTSB = -ds['Ibetav'] - ds['Istretchp'] + ds['Ires']

	F_Ti = Tdecorrw(F.values)
	R_TSB_Ti = Tdecorrw(R_TSB.values)
	R_tTSB_Ti = Tdecorrw(R_tTSB.values)

# Integral timescales for the circumpolar average of F and R.
# Choose the conservative value of 5 days for all -> Neff = T/Ti = 1825 days/5 days = 365 EDoF.*****
#
# In [36]: F_Ti
# Out[36]: 2.6560768522084586
#
# In [37]: R_TSB_Ti
# Out[37]: 2.291490519265456
#
# In [38]: R_tTSB_Ti
# Out[38]: 0.8437070018333408

# Cross-correlations between terms and autocorrelations.
nblks_corr = 100

fnames = [head_data+'circulation_terms-Amundsen-Bellingshausen.nc',
          head_data+'circulation_terms-WAP.nc',
          head_data+'circulation_terms-Weddell.nc',
          head_data+'circulation_terms-W-EA.nc',
          head_data+'circulation_terms-E-EA.nc',
          head_data+'circulation_terms-Ross.nc',
          head_data+'circulation_terms_circumpolar.nc',]

print("")

n=0
for fname in fnames:
  print(fname,"*******************************")
  print("****************************************")
  ds = open_dataset(fname)
  segment = fname.split('terms')[-1].split('.')[0][1:]

  F = ds['Icurlvdiff'].values
  TSB = -ds['Ibetav'].values - ds['Istretchp'].values # +beta*V +f*w_I, on LHS.
  tTSB = -ds['Ibetav'].values - ds['Istretchp'].values + ds['Ires'].values # +beta*V +f*w_I + dzeta/dt, on LHS.

  # Test Topographic Sverdrup Balance.
  R = TSB
  lags, FRcorr = crosscorr(F, R, nblks_corr, onesided=False, verbose=False)
  fmaxcorr = np.where(lags==0)[0][0]
  print("")
  print("TSB: Zero-lag correlation of 'Icurlvvdiff' with (beta*V + f*wI) is %.2f."%FRcorr[fmaxcorr])

  # Test Transient Topographic Sverdrup Balance.
  R = tTSB
  lags, FRcorr = crosscorr(F, R, nblks_corr, onesided=False, verbose=False)
  fmaxcorr = np.where(lags==0)[0][0]
  print("tTSB: Zero-lag correlation of 'Icurlvvdiff' with (beta*V + f*wI + dzeta/dt) is %.2f."%FRcorr[fmaxcorr])
  print("")

  n+=1


../../data_reproduce_figs/circulation_terms-Amundsen-Bellingshausen.nc *******************************
****************************************


/home/andre/anaconda3/envs/py3/lib/python3.7/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/andre/anaconda3/envs/py3/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)



TSB: Zero-lag correlation of 'Icurlvvdiff' with (beta*V + f*wI) is 0.54.
tTSB: Zero-lag correlation of 'Icurlvvdiff' with (beta*V + f*wI + dzeta/dt) is 0.94.

../../data_reproduce_figs/circulation_terms-WAP.nc *******************************
****************************************


/home/andre/anaconda3/envs/py3/lib/python3.7/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/andre/anaconda3/envs/py3/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)



TSB: Zero-lag correlation of 'Icurlvvdiff' with (beta*V + f*wI) is 0.18.
tTSB: Zero-lag correlation of 'Icurlvvdiff' with (beta*V + f*wI + dzeta/dt) is 0.11.

../../data_reproduce_figs/circulation_terms-Weddell.nc *******************************
****************************************


...


TSB: Zero-lag correlation of 'Icurlvvdiff' with (beta*V + f*wI) is 0.26.
tTSB: Zero-lag correlation of 'Icurlvvdiff' with (beta*V + f*wI + dzeta/dt) is 0.63.

../../data_reproduce_figs/circulation_terms-Ross.nc *******************************
****************************************


/home/andre/anaconda3/envs/py3/lib/python3.7/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/andre/anaconda3/envs/py3/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)



TSB: Zero-lag correlation of 'Icurlvvdiff' with (beta*V + f*wI) is 0.19.
tTSB: Zero-lag correlation of 'Icurlvvdiff' with (beta*V + f*wI + dzeta/dt) is 0.48.

../../data_reproduce_figs/circulation_terms_circumpolar.nc *******************************
****************************************


/home/andre/anaconda3/envs/py3/lib/python3.7/site-packages/numpy/lib/function_base.py:2526: RuntimeWarning: Degrees of freedom <= 0 for slice
  c = cov(x, y, rowvar)
/home/andre/anaconda3/envs/py3/lib/python3.7/site-packages/numpy/lib/function_base.py:2455: RuntimeWarning: divide by zero encountered in true_divide
  c *= np.true_divide(1, fact)



TSB: Zero-lag correlation of 'Icurlvvdiff' with (beta*V + f*wI) is 0.40.
tTSB: Zero-lag correlation of 'Icurlvvdiff' with (beta*V + f*wI + dzeta/dt) is 0.61.

